# NLP API tutorial

This tutorial demonstates how to use Python Riva API.

## <font color="blue">Server</font>

Before running client part of Riva, please set up a server. The simplest
way to do this is to follow
[quick start guide](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/quick-start-guide.html#local-deployment-using-quick-start-scripts).


## <font color="blue">Authentication</font>

Before using Riva services you will need to establish connection with a server.

In [ ]:
import riva_api

uri = "localhost:50051"  # Default value

auth = riva_api.Auth(uri=uri)

## <font color="blue">Setting up service</font>

To instantiate a service pass `riva_api.Auth` instance to a constructor.

In [ ]:
nlp_service = riva_api.NLPService(auth)

You may find full response field description in [documentation](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/reference/protos/riva_asr.proto.html?highlight=max%20alternatives#riva-proto-riva-asr-proto).

In [ ]:
from riva_api.proto.riva_nlp_pb2 import (
    TextClassResponse,
    TokenClassResponse,
    TextTransformResponse,
    NaturalQueryResponse,
)

## <font color="blue">Text classification</font>

In [ ]:
queries = ["A hurricane is approaching Japan.", "What is the weather on Wednesday in Moscow?"]
model = "riva_intent_weather"

In [ ]:
response: TextClassResponse = nlp_service.classify_text(queries, model)

In [ ]:
print(response)

In [ ]:
detected_intent = response.results[0].labels[0].class_name

In [ ]:
print(detected_intent)

You may use a function `riva_api.extract_most_probable_text_class_and_confidence()`. The function returns a list of most probable intents and their scores for all queries.

In [ ]:
classes, probs = riva_api.extract_most_probable_text_class_and_confidence(response)

In [ ]:
print(classes)
print(probs)

## <font color="blue">Token classification</font>

In [ ]:
response: TokenClassResponse = nlp_service.classify_tokens(queries[1], model)

In [ ]:
print(response)

In [ ]:
token = response.results[0].results[0].token
class_name = response.results[0].results[0].label[0].class_name
class_score = response.results[0].results[0].label[0].score

In [ ]:
print(token, class_name, class_score)

You may use a function `riva_api.extract_most_probable_token_classification_predictions()`. The function returns:
 - list of tokens lists for all elements of a batch,
 - list of most probable classes lists for all elements of a batch,
 - list of most probable classes confidences lits for all elements of a batch,
 - list of token span starts lists for all elements of a batch,
 - list of token span ends lists for all elements of a batch.

In [ ]:
tokens, class_names, confidences, starts, ends = riva_api.extract_most_probable_token_classification_predictions(response)

In [ ]:
print("First batch element tokens:", tokens[0])
print("First batch element first token class name:", class_names[0][0])
print(confidences)
print(starts)
print(ends)

> Spans do not work properly for batches which contain more than 1 element.

## <font color="blue">Punctuation and Capitalization</font>

In [ ]:
queries = [
    "by the early 20th century the gar complained more and more about the younger generation",
    "boa Vista is the capital of the brazilian state of roraima situated on the western bank of "
    "the branco river the city lies 220 km from brazil's border with venezuela.",
]
model_name = "riva-punctuation-en-US"
response: TextTransformResponse = nlp_service.punctuate_text(queries, model_name)

In [ ]:
print(response)

In [ ]:
first_query_result = response.text[0]
print(first_query_result)

## <font color="blue">Question answering</font>

In [ ]:
query = "How many gigatons of carbon dioxide was released in 2005?"
context = (
    "In 2010 the Amazon rainforest experienced another severe drought, in some ways more extreme than the "
    "2005 drought. The affected region was approximate 1,160,000 square miles (3,000,000 km2) of "
    "rainforest, compared to 734,000 square miles (1,900,000 km2) in 2005. The 2010 drought had three "
    "epicenters where vegetation died off, whereas in 2005 the drought was focused on the southwestern "
    "part. The findings were published in the journal Science. In a typical year the Amazon absorbs 1.5 "
    "gigatons of carbon dioxide; during 2005 instead 5 gigatons were released and in 2010 8 gigatons were "
    "released."
)
response: NaturalQueryResponse = nlp_service.natural_query(query, context)

In [ ]:
print(response)

In [ ]:
answer = response.results[0].answer
score = response.results[0].score

In [ ]:
print(answer)

In [ ]:
print(score)